### Haystack Integration

[Haystack](https://haystack.deepset.ai/) is the open-source Python framework developed by deepset. Its modular design allows users to implement custom pipelines to build production-ready LLM applications, like retrieval-augmented generative pipelines and state-of-the-art search systems. It integrates with Hugging Face Transformers, Elasticsearch, OpenSearch, OpenAI, Cohere, Anthropic and others, making it an extremely popular framework for teams of all sizes.

https://langfuse.com/docs/integrations/haystack/example-python

### Installation and Setup

In [ ]:
# install haystack, langfuse, and the langfuse-haystack integration package
%pip install haystack-ai langfuse-haystack "langfuse<3.0.0"

# additional requirements for this cookbook
%pip install sentence-transformers datasets mwparserfromhell

In [1]:
import os
 
# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-962190cc-b2bd-47c0-b752-8de287a2a5c1" 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-4e9959c3-0935-4142-b789-734beb81d15a" 
LANGFUSE_HOST="http://localhost:3000"

# Your openai key
os.environ["OPENAI_API_KEY"] = "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"

# Enable Haystack content tracing
os.environ["HAYSTACK_CONTENT_TRACING_ENABLED"] = "True"

### Basic RAG Pipeline
Building a basic retrieval-augmented generative (RAG) pipeline. First you’ll load your data to the Document Store, then connect components together into a RAG pipeline, and finally ask a question.

In [2]:
from datasets import load_dataset
from haystack import Document, Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.connectors.langfuse import LangfuseConnector

In [ ]:
def get_pipeline(document_store: InMemoryDocumentStore):
    retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=2)
 
    # A prompt corresponds to an NLP task and contains instructions for the model. Here, the pipeline will go through each Document to figure out the answer.
    template = """
    Given the following information, answer the question.
    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}
    Question: {{question}}
    Answer:
    """
 
    prompt_builder = PromptBuilder(template=template)
 
    basic_rag_pipeline = Pipeline()
    # Add components to your pipeline
    basic_rag_pipeline.add_component("tracer", LangfuseConnector("Basic RAG Pipeline"))
    basic_rag_pipeline.add_component(
        "text_embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
    )
    basic_rag_pipeline.add_component("retriever", retriever)
    basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
    # 当前问题: 使用 OpenRouter API key 调用 OpenAI 官方 API
    #basic_rag_pipeline.add_component("llm", OpenAIGenerator(model="gpt-4o", generation_kwargs={"n": 2}))

    # 使用 OpenRouter 兼容的配置
    basic_rag_pipeline.add_component("llm", OpenAIGenerator(
        api_base_url="https://openrouter.ai/api/v1",
        model="deepseek/deepseek-chat",  # 使用 OpenRouter 支持的模型
        generation_kwargs={"n": 1}  # 减少并发请求
    ))

 
    # Now, connect the components to each other
    # NOTE: the tracer component doesn't need to be connected to anything in order to work
    basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
    basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
    basic_rag_pipeline.connect("prompt_builder", "llm")
 
    return basic_rag_pipeline

Then we load data into DocumentStore. In this example, we use the trivia_qa_tiny dataset.

In [ ]:
document_store = InMemoryDocumentStore()
dataset = load_dataset("SpeedOfMagic/trivia_qa_tiny", split="train")
embedder = SentenceTransformersDocumentEmbedder("sentence-transformers/all-MiniLM-L6-v2")
embedder.warm_up()
 
docs_with_embeddings = []
for entry in dataset:
    # Create a Document object for each entry, handling the question (str) and answer (str) data correctly
    content = f"Question: {entry['question']} Answer: {entry['answer']}"
    doc = Document(content=content)
 
    # Embed the document using the embedder
    # Only takes in list of Documents
    embedder.run([doc])
 
    # Collect the embedded documents
    docs_with_embeddings.append(doc)
 
# Write the embedded documents to the document store
document_store.write_documents(docs_with_embeddings)

Then ask a question based on the data we loaded in.

In [ ]:
pipeline = get_pipeline(document_store)
question = "What can you tell me about Truman Capote?"
response = pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

In [35]:
pipeline.draw("./haystack_pipeline.png")

In [ ]:
print("Trace url:", response["tracer"]["trace_url"])
print("Response:", response["llm"]["replies"][0])

### Add score to the trace


当前环境安装的是 langfuse 2.x，所以无法导入和使用 get_client。

In [17]:
# from langfuse import get_client
 
# langfuse = get_client()
 
# trace_id = langfuse.get_current_trace_id()
 
# langfuse.create_score(
#     trace_id=trace_id,
#     name="quality",
#     value=1,
#     comment="Cordial and relevant", # optional
# )


from langfuse import Langfuse

langfuse = Langfuse(
    public_key=os.environ["LANGFUSE_PUBLIC_KEY"],
    secret_key=os.environ["LANGFUSE_SECRET_KEY"],
    host=os.environ.get("LANGFUSE_HOST", "http://localhost:3000")
)

# 从 response 获取 trace_id
trace_url = response["tracer"]["trace_url"]
trace_id = trace_url.split("/traces/")[-1].split("?")[0]


langfuse.score(
    trace_id=trace_id,
    name="quality",
    value=1,
    comment="Cordial and relevant"
)